In [1]:
from collections import namedtuple

In [2]:
Markpat = namedtuple("Markpat", [ "last_turn", "turn", "empty" ])

In [3]:
Markpat = namedtuple("Markpat", "last_turn turn empty")
Markpat = namedtuple("Markpat", "last_turn, turn, empty")

In [4]:
# typename の名前がアラビア数字で始まっている
namedtuple("0Markpat", ["last_turn", "turn", "empty"]) 

ValueError: Type names and field names must be valid identifiers: '0Markpat'

In [5]:
namedtuple("Markpat", ["_last_turn", "turn", "empty"]) 

ValueError: Field names cannot start with an underscore: '_last_turn'

In [6]:
print(Markpat(2, 0, 1))
print(Markpat(2, 0, empty=1))
print(Markpat(2, turn=0, empty=1))
print(Markpat(last_turn=2, turn=0, empty=1))
print(Markpat(empty=1, last_turn=2, turn=0))

Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)


In [7]:
Markpat3 = namedtuple("Markpat2", [ "last_turn", "turn", "empty" ])
print(Markpat3(0, 1, 2))

Markpat2(last_turn=0, turn=1, empty=2)


In [8]:
print(Markpat(0, 1))

TypeError: Markpat.__new__() missing 1 required positional argument: 'empty'

In [9]:
print(Markpat(0, 1, 2, 3))

TypeError: Markpat.__new__() takes 4 positional arguments but 5 were given

In [10]:
Point = namedtuple("Point", ["x", "y"], defaults=[1, 2])

# 実引数が記述されていないので、両方の要素が、デフォルト値である 1 と 2 に設定される
print(Point())       
# 1 つ目の要素が実引数に記述された 3 に、2 つ目の要素がデフォルト値である 2 に設定される
print(Point(3))      
# 2 つの要素が、実引数に記述された 4 と 5 に設定される
print(Point(4, 5))   

Point(x=1, y=2)
Point(x=3, y=2)
Point(x=4, y=5)


In [11]:
Point3D = namedtuple("Point3D", ["x", "y", "z"], defaults=[1, 2])

print(Point3D(3))
print(Point3D(4, 5))
print(Point3D(6, 7, 8))

Point3D(x=3, y=1, z=2)
Point3D(x=4, y=5, z=2)
Point3D(x=6, y=7, z=8)


In [12]:
print(Point3D())

TypeError: Point3D.__new__() missing 1 required positional argument: 'x'

In [13]:
mp = Markpat(2, 0, 1)
print(mp[0], mp[1], mp[2])

2 0 1


In [14]:
mp = Markpat(2, 0, 1)
last_turn, turn, empty = mp
print(last_turn, turn, empty)

2 0 1


In [15]:
mp = Markpat(2, 0, 1)
print(mp.last_turn, mp.turn, mp.empty)

2 0 1


In [16]:
mp = Markpat(2, 0, 1)
print(mp["last_turn"])

TypeError: tuple indices must be integers or slices, not str

In [17]:
mp = Markpat(2, 0, 1)
name = "last_turn"
print(getattr(mp, name))

2


In [18]:
from marubatsu import Marubatsu
from collections import defaultdict

def count_marks(self, coord, dx, dy, datatype="dict"):     
    x, y = coord   
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.board[x][y]] += 1
        x += dx
        y += dy

    if datatype == "dict":
        return count
    else:
        return Markpat(count[self.last_turn], count[self.turn], count[Marubatsu.EMPTY])

Marubatsu.count_marks = count_marks

In [19]:
from ai import ai_match, ai2, ai7s, ai8s

ai_match(ai=[ai8s, ai2])

ai8s VS ai2
count     win    lose    draw
o        9843      11     146
x        8926     234     840
total   18769     245     986

ratio     win    lose    draw
o       98.4%    0.1%    1.5%
x       89.3%    2.3%    8.4%
total   93.8%    1.2%    4.9%



In [20]:
ai_match(ai=[ai8s, ai7s])

ai8s VS ai7s
count     win    lose    draw
o        3313     393    6294
x         377    2923    6700
total    3690    3316   12994

ratio     win    lose    draw
o       33.1%    3.9%   62.9%
x        3.8%   29.2%   67.0%
total   18.4%   16.6%   65.0%



In [21]:
from ai import ai_by_score

def ai8s(mb, debug=False):
    def eval_func(mb):
        # 真ん中のマスに着手している場合は、評価値として 2 を返す
        if mb.last_move == (1, 1):
            return 3
    
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 2

        markpats = mb.enum_markpats()
        # 相手が勝利できる場合は評価値として -1 を返す
        if Markpat(last_turn=0, turn=2, empty=1) in markpats:
            return -1
        # 次の自分の手番で自分が勝利できる場合は評価値として 1 を返す
        elif Markpat(last_turn=2, turn=0, empty=1) in markpats:
            return 1
        # それ以外の場合は評価値として 0 を返す
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug) 

In [22]:
ai_match(ai=[ai8s, ai2])

ai8s VS ai2


count     win    lose    draw
o        9820       9     171
x        8910     245     845
total   18730     254    1016

ratio     win    lose    draw
o       98.2%    0.1%    1.7%
x       89.1%    2.5%    8.5%
total   93.7%    1.3%    5.1%



In [23]:
ai_match(ai=[ai8s, ai7s])

ai8s VS ai7s
count     win    lose    draw
o        3317     431    6252
x         386    2961    6653
total    3703    3392   12905

ratio     win    lose    draw
o       33.2%    4.3%   62.5%
x        3.9%   29.6%   66.5%
total   18.5%   17.0%   64.5%



In [24]:
from typing import NamedTuple

In [25]:
class Markpat(NamedTuple):   
    last_turn: int
    turn: int
    empty: int

In [26]:
class Markpat(NamedTuple):
    """マークのパターン.
    
    Attributes:
        last_turn (int):
            直前のターンのマークの数
        turn (int):
            現在のターンのマークの数
        empty (int):
            空のマスの数
    """
    
    last_turn: int
    turn: int
    empty: int

In [27]:
print(Markpat(2, 0, 1))
print(Markpat(2, 0, empty=1))
print(Markpat(2, turn=0, empty=1))
print(Markpat(last_turn=2, turn=0, empty=1))
print(Markpat(empty=1, last_turn=2, turn=0))

Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)
Markpat(last_turn=2, turn=0, empty=1)


In [28]:
ai_match(ai=[ai8s, ai2])

ai8s VS ai2
count     win    lose    draw
o        9845       6     149
x        8930     236     834
total   18775     242     983

ratio     win    lose    draw
o       98.5%    0.1%    1.5%
x       89.3%    2.4%    8.3%
total   93.9%    1.2%    4.9%



In [29]:
ai_match(ai=[ai8s, ai7s])

ai8s VS ai7s
count     win    lose    draw
o        3319     415    6266
x         391    2939    6670
total    3710    3354   12936

ratio     win    lose    draw
o       33.2%    4.2%   62.7%
x        3.9%   29.4%   66.7%
total   18.6%   16.8%   64.7%



In [30]:
class Point3D(NamedTuple):
    last_turn: int
    turn: int = 1
    empty: int = 2

print(Point3D(3))
print(Point3D(4, 5))
print(Point3D(6, 7, 8))

Point3D(last_turn=3, turn=1, empty=2)
Point3D(last_turn=4, turn=5, empty=2)
Point3D(last_turn=6, turn=7, empty=8)


In [31]:
print(Markpat("a", "b", "c"))

Markpat(last_turn='a', turn='b', empty='c')


In [32]:
a: int = 1 # グローバル変数の型アノテーション

def f():
    b : float = 1.2  # ローカル変数の型アノテーション
    
class C:
    d : str = "text"  # クラス属性の型アノテーション
    
    def __init__(self):
        self.e : list[int] = [1, 2, 3]  # インスタンスの属性の型アノテーション

In [33]:
x

NameError: name 'x' is not defined

In [34]:
x: int

In [35]:
print(x)

NameError: name 'x' is not defined

In [36]:
class Point(NamedTuple):   
    x
    y

NameError: name 'x' is not defined

In [37]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [38]:
p = Point(1, 2)
print(p.x, p.y)

1 2


In [39]:
p = Point(1, 2)
print(p)

In [40]:
p1 = Point(1, 2)
p2 = Point(1, 2)
print(p1 == p2)

False


In [41]:
from dataclasses import dataclass

In [42]:
@dataclass
class Point:
    x: int
    y: int

In [43]:
p = Point(1, 2)
print(p)

Point(x=1, y=2)


In [44]:
p1 = Point(1, 2)
p2 = Point(1, 2)
p3 = Point(3, 4)
print(p1 == p2)
print(p1 == p3)

True
False


In [45]:
p = Point(1, 2)
p.z = 3
print(p)

Point(x=1, y=2)


In [46]:
p1 = Point(1, 2)
p2 = Point(1, 2)
p1.z = 3
p2.z = 5
print(p1 == p2)

True


In [47]:
@dataclass
class Point:
    x: int
    y: int

    def __init__(self):
        pass

In [48]:
p = Point(1, 2)

TypeError: Point.__init__() takes 1 positional argument but 3 were given

In [49]:
p = Point()
print(p.x)

AttributeError: 'Point' object has no attribute 'x'

In [50]:
@dataclass
class Point:
    x: int
    y: int

    def __post_init__(self):
        print("post init")

In [51]:
print(Point(1, 2))

post init
Point(x=1, y=2)


In [52]:
print({Point(1, 2)})

post init


TypeError: unhashable type: 'Point'

In [53]:
p = Point(1, 2)
x, y = p

post init


TypeError: cannot unpack non-iterable Point object

In [54]:
@dataclass(frozen=True)
class Point:
    x: int
    y: int

print({Point(1, 2)})

{Point(x=1, y=2)}


In [55]:
def ai6s(mb, debug=False):
    def eval_func(mb):
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 1

        markpats = mb.enum_markpats()
        # 相手が勝利できる場合は評価値として -1 を返す
        if Markpat(last_turn=0, turn=2, empty=1) in markpats:
            return -1
        # それ以外の場合は評価値として 0 を返す
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)  

In [56]:
from ai import ai6, ai7

ai_match(ai=[ai6s, ai6])

ai6s VS ai6
count     win    lose    draw
o        3140    1702    5158
x        1704    3116    5180
total    4844    4818   10338

ratio     win    lose    draw
o       31.4%   17.0%   51.6%
x       17.0%   31.2%   51.8%
total   24.2%   24.1%   51.7%



In [57]:
def ai7s(mb, debug=False):
    def eval_func(mb):
        # 真ん中のマスに着手している場合は、評価値として 2 を返す
        if mb.last_move == (1, 1):
            return 2
            
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 1

        markpats = mb.enum_markpats()
        # 相手が勝利できる場合は評価値として -1 を返す
        if Markpat(last_turn=0, turn=2, empty=1) in markpats:
            return -1
        # それ以外の場合は評価値として 0 を返す
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)  

In [58]:
ai_match(ai=[ai7s, ai7])

ai7s VS ai7
count     win    lose    draw
o        2962     406    6632
x         392    2933    6675
total    3354    3339   13307

ratio     win    lose    draw
o       29.6%    4.1%   66.3%
x        3.9%   29.3%   66.8%
total   16.8%   16.7%   66.5%

